# lab 6 - servers simulation
- Ricardo Méndez 21289
- Sara Echeverría 21371
- Melissa Pérez 21385

Suponga que usted está a cargo de definir la arquitectura para el lanzamiento de su próxima aplicación web, C3 (sistema de contabilidad de la carreta contadora). La junta directiva le ha solicitado encontrar el mejor servicio de hosting para el proyecto. Tras una extensa investigación, concluye que las mejores opciones son: 
- Proveedor 1 - Mountain Mega Computing, con una infraestructura de servidor único que puede atender hasta 100 solicitudes por segundo.
- Proveedor 2 - Pizzita Computing, con una infraestructura en la nube de múltiples servidores medianamente potentes, donde cada servidor tiene una décima parte de la potencia del servidor de Mountain Mega Computing, y se paga solo por los servidores necesarios. 

Las pruebas de estrés y las proyecciones indican que su aplicación no excederá las 2,400 solicitudes por minuto durante los primeros dos años. Un análisis de sistemas similares sugiere que las solicitudes seguirán un proceso de Poisson, con tiempos de servicio modelados por una variable exponencial. Como debe presentar su decisión mañana, opta por realizar una simulación basada en la promoción de los proveedores para concluir cuál será la mejor opción.

In [1]:
import numpy as np
from scipy.stats import expon

In [2]:
def simulateProvider1(lambdaRate, serviceRate, simulationTime):
    # converting simulationTime from hours to seconds
    totalSimulationSeconds = simulationTime * 3600
    
    # calculate inter-arrival and service times for provider 1
    interArrivalTimes = expon.rvs(scale=1/lambdaRate, size=int(lambdaRate * totalSimulationSeconds))
    serviceTimes = expon.rvs(scale=1/serviceRate, size=int(lambdaRate * totalSimulationSeconds))

    # initializing simulation parameters
    currentTime = 0
    lastServiceEndTime = 0
    totalWaitTime = 0
    totalBusyTime = 0

    # processing each request
    for interArrival, serviceTime in zip(interArrivalTimes, serviceTimes):
        currentTime += interArrival
        if currentTime > lastServiceEndTime:
            waitTime = 0
            serviceStartTime = currentTime
        else:
            waitTime = lastServiceEndTime - currentTime
            serviceStartTime = lastServiceEndTime

        serviceEndTime = serviceStartTime + serviceTime
        lastServiceEndTime = serviceEndTime
        totalWaitTime += waitTime
        totalBusyTime += serviceTime

    # calculating metrics
    averageWaitTime = totalWaitTime / len(interArrivalTimes)
    utilization = totalBusyTime / totalSimulationSeconds
    return averageWaitTime, utilization

In [3]:
def simulateProvider2(lambdaRate, serverPower, numServers, simulationTime):
    # converting simulationTime from hours to seconds
    totalSimulationSeconds = simulationTime * 3600

    # calculate inter-arrival and service times for provider 2
    interArrivalTimes = expon.rvs(scale=1/lambdaRate, size=int(lambdaRate * totalSimulationSeconds))
    serviceRate = serverPower * numServers
    serviceTimes = expon.rvs(scale=1/serviceRate, size=int(lambdaRate * totalSimulationSeconds))

    # initializing simulation parameters
    currentTime = 0
    serversEndTime = [0] * numServers
    totalWaitTime = 0
    totalBusyTime = 0

    # processing each request
    for interArrival, serviceTime in zip(interArrivalTimes, serviceTimes):
        currentTime += interArrival
        earliestServer = np.argmin(serversEndTime)
        if currentTime > serversEndTime[earliestServer]:
            waitTime = 0
            serviceStartTime = currentTime
        else:
            waitTime = serversEndTime[earliestServer] - currentTime
            serviceStartTime = serversEndTime[earliestServer]

        serviceEndTime = serviceStartTime + serviceTime
        serversEndTime[earliestServer] = serviceEndTime
        totalWaitTime += waitTime
        totalBusyTime += serviceTime

    # calculating metrics
    averageWaitTime = totalWaitTime / len(interArrivalTimes)
    utilization = totalBusyTime / (totalSimulationSeconds * numServers)
    return averageWaitTime, utilization

In [4]:
# simulation parameters
lambdaRate = 40
serverPower = 10  
numServers = 5  
simulationTime = 1

# simulate each provider
averageWaitTime1, utilization1 = simulateProvider1(lambdaRate, 100, simulationTime)
averageWaitTime2, utilization2 = simulateProvider2(lambdaRate, serverPower, numServers, simulationTime)

# print results
print("-> provider 1 - average wait time: {:.4f} seconds, utilization: {:.2%}".format(averageWaitTime1, utilization1))
print("-> provider 2 - average wait time: {:.4f} seconds, utilization: {:.2%}".format(averageWaitTime2, utilization2))

-> provider 1 - average wait time: 0.0065 seconds, utilization: 39.89%
-> provider 2 - average wait time: 0.0000 seconds, utilization: 16.02%


## Tasks
1. Modele, simule y analice el comportamiento de ambos sistemas durante una hora de ejecución de C3, y para cada sistema responda
- ¿Cuántas solicitudes atendió cada servidor?
- ¿Cuánto tiempo estuvo cada servidor ocupado?
- ¿Cuánto tiempo estuvo cada servidor desocupado (iddle)?
- ¿Cuánto tiempo en total estuvieron las solicitudes en cola?
- En promedio ¿cuánto tiempo estuvo cada solicitud en cola?
- En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?
- ¿Cuál es el momento de la salida de la última solicitud?

2. Determine empíricamente cuántos servidores se necesitaría “alquilar” en Pizzita computing para asegurar que siempre habrá al menos un servidor disponible para atender una solicitud dada (en otras palabras, una solicitud nunca tiene que esperar en cola)

3. Se espera que a partir del tercer año del lanzamiento de su aplicación, la cantidad de usuarios sufra un alza, y por tanto deberán atender como máximo 6000 solicitudes por minuto. Resuelva el inciso 1 y 2 para esta nueva configuración.

4. Emita una recomendación para la junta directiva